## SRJ - No consecutive operators

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)

ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/4
operator_check = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 1.62
symbols = ["Li","H"]
electrons = 4
orbitals = 12
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[3840] = 1.0 # Every molecule change, you need to change this index #3840
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the ash_excitationis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    #For generating the new state, they are adding the excitation in the exponent of the rotation, thats 
    #why we need to use qml.FermionicSE or qml.FermionicDE
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=theta, wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
     
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

            #print('D-Generating functions are', super2)
    #print('Total len of Generating functions is', len(generatingfns))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    print('The length of operator pool', len(operator_pool))
    max_operator = None

    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
        
        for i in operator_pool:
            if j == 1:
            # First iteration: allow all operators
                w = qml.fermi.jordan_wigner(i)
                if np.array_equal(k, hf_state):
                    current_value = abs(2*(commutator_0(H, w, k)))
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))
                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            else:
            # Block only the last used operator (avoid consecutive selection)
                if operator_check and i == operator_check[-1]:
                    continue  # Skip if operator is the same as last used
                w = qml.fermi.jordan_wigner(i)
                if np.array_equal(k, hf_state):
                    current_value = abs(2*(commutator_0(H, w, k)))
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))
                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")
        operator_check.append(max_operator)
        print('The last saved operator check is', operator_check[-1])
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after newly added Generating function is', counts)
        
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
            counts_ind = measure(states[-1])
            print('The individula states counts', counts_ind)
        else:
            print('Skipping individual excitation state for j < 2')
        states.append(ostate)
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H)
        #print('Ham matrix is', Ham_matrix)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                #print('The value of i and j is', i, j)
                S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        #print(f'S matrix with {i} and {j} is', S)
        #print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy)    
    return ash_excitation, states,eig,gs_energy, Ham_matrix

In [2]:
ash_excitation, states,eig,gs_energy, Ham_matrix = adaptvqe(adapt_it=5, e_th=1e-12)
print('The gs energy:', gs_energy)  
print('Ash excitation is:', ash_excitation)

HF state is [6.123234e-17+0.j 0.000000e+00+0.j 0.000000e+00+0.j ... 0.000000e+00+0.j
 0.000000e+00+0.j 0.000000e+00+0.j]
HF state is -7.861149428178474
The length of operator pool 92
The adapt iteration now is 1
The highest operator value is 0.24829589380807315 for operator a⁺(2) a⁺(3) a(10) a(11)
The last saved operator check is a⁺(2) a⁺(3) a(10) a(11)
Highest gradient excitation is [2, 3, 10, 11]
ash_excitation is [[2, 3, 10, 11]]
The length of ash_excitation before generating matrix is 1
 0: ─╭|Ψ⟩──────────────────────────────────┤  State
 1: ─├|Ψ⟩──────────────────────────────────┤  State
 2: ─├|Ψ⟩─╭FermionicDoubleExcitation(0.79)─┤  State
 3: ─├|Ψ⟩─├FermionicDoubleExcitation(0.79)─┤  State
 4: ─├|Ψ⟩─│────────────────────────────────┤  State
 5: ─├|Ψ⟩─│────────────────────────────────┤  State
 6: ─├|Ψ⟩─│────────────────────────────────┤  State
 7: ─├|Ψ⟩─│────────────────────────────────┤  State
 8: ─├|Ψ⟩─│────────────────────────────────┤  State
 9: ─├|Ψ⟩─│─────────────────────────